<a href="https://colab.research.google.com/github/CryptoRobotFr/General-code-trading-bot/blob/main/Indicateurs_volatilit%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta python-binance

In [ ]:
import pandas as pd
from binance.client import Client
import ta
import numpy as np

# Variables pour les données
client = Client()
pair_symbol = "BTCUSDT"
time_interval = Client.KLINE_INTERVAL_1DAY
start_date = "01 january 2017"

# Récupération des données
klinesT = client.get_historical_klines(pair_symbol, time_interval, start_date)

# Créer un tableau grâce aux données
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

# Supprime les colonnes inutiles
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# Convertit les colonnes en numéric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

# Convertit les dates dans un format lisible
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

# Affiche le tableau
df

,open,high,low,close,volume
timestamp,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...
2022-02-26,39219.16,40348.45,38573.18,39116.72,29361.256800
2022-02-27,39116.73,39855.70,37000.00,37699.07,46229.447190
2022-02-28,37699.08,44225.84,37450.17,43160.00,73945.638580


In [ ]:
E#Court code pour définir les différents indicateurs de volatilité à implementer dans vos stratégies de trading

# Kaufman ou KAMA
df["KAMA"] = ta.momentum.kama(df['close'], window=10, pow1=2, pow2=30)
# ------------------------- #


# ATR
df["ATR"] = ta.volatility.average_true_range(df['high'],df['low'],df['close'],window=2)
# ------------------------- #


# Les bandes de Bollingers
df["BOL_HIGH"] = ta.volatility.bollinger_hband(df['close'],window=20R,window_dev=2)
df["BOL_LOW"] = ta.volatility.bollinger_lband(df['close'],window=20,window_dev=2)
df["BOL_MEDIUM"] = ta.volatility.bollinger_mavg(df['close'],window=20)
df["BOL_GAP"] = ta.volatility.bollinger_wband(df['close'],window=20,window_dev=2)
# Return binaire 0 ou 1 
df["BOL_HIGHER"] = ta.volatility.bollinger_hband_indicator(df['close'],window=20,window_dev=2)
df["BOL_LOWER"] = ta.volatility.bollinger_lband_indicator(df['close'],window=20,window_dev=2)
# ------------------------- #


# Le canal de Donchian
df["DON_HIGH"] = ta.volatility.donchian_channel_hband(df['high'],df['low'],df['close'],window=20,offset=0)
df["DON_LOW"] = ta.volatility.donchian_channel_lband(df['high'],df['low'],df['close'],window=20,offset=0)
df["DON_MEDIUM"] = ta.volatility.donchian_channel_mband(df['high'],df['low'],df['close'],window=20,offset=0)
# ------------------------- #


# Le canal de Keltner 
df["KEL_HIGH"] = ta.volatility.keltner_channel_hband(df['high'],df['low'],df['close'],window=20,window_atr=10)
df["KEL_LOW"] = ta.volatility.keltner_channel_lband(df['high'],df['low'],df['close'],window=20,window_atr=10)
df["KEL_MEDIUM"] = ta.volatility.keltner_channel_mband(df['high'],df['low'],df['close'],window=20,window_atr=10)
# Return binaire 0 ou 1 
df["KEL_HIGHER"] = ta.volatility.keltner_channel_hband_indicator(df['high'],df['low'],df['close'],window=20,window_atr=10)
df["KEL_LOWER"] = ta.volatility.keltner_channel_lband_indicator(df['high'],df['low'],df['close'],window=20,window_atr=10)
# ------------------------- #


# Chopiness
def chop(high, low, close, window=14):
    ''' Choppiness indicator
    '''
    tr1 = pd.DataFrame(high - low).rename(columns={0: 'tr1'})
    tr2 = pd.DataFrame(abs(high - close.shift(1))
                       ).rename(columns={0: 'tr2'})
    tr3 = pd.DataFrame(abs(low - close.shift(1))
                       ).rename(columns={0: 'tr3'})
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis=1, join='inner').dropna().max(axis=1)
    atr = tr.rolling(1).mean()
    highh = high.rolling(window).max()
    lowl = low.rolling(window).min()
    chop_serie = 100 * np.log10((atr.rolling(window).sum()) /
                          (highh - lowl)) / np.log10(window)
    return pd.Series(chop_serie, name="CHOP")

df["CHOP"] = chop(df['high'],df['low'],df['close'],window=14)
# ------------------------- #

df

,open,high,low,close,volume,KAMA,ATR,BOL_HIGH,BOL_LOW,BOL_MEDIUM,BOL_GAP,BOL_HIGHER,BOL_LOWER,DON_HIGH,DON_LOW,DON_MEDIUM,KEL_HIGH,KEL_LOW,KEL_MEDIUM,KEL_HIGHER,KEL_LOWER,CHOP
timestamp,,,,,,,,,,,,,,,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4608.386667,4039.086667,NaN,0.0,0.0,NaN
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,358.700000,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4590.345000,3872.945000,NaN,0.0,0.0,NaN
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,NaN,346.695000,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4524.534444,3823.141111,NaN,0.0,0.0,NaN
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,NaN,262.577500,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4465.515000,3850.240000,NaN,0.0,0.0,NaN
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,NaN,235.203750,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4417.138667,3841.786667,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-26,39219.16,40348.45,38573.18,39116.72,29361.256800,39884.168418,2268.595010,46032.307699,36113.548301,41072.9280,24.149141,0.0,0.0,45821.0,34322.28,40071.64,43320.046667,38958.671667,41139.359167,0.0,0.0,40.329925
2022-02-27,39116.73,39855.70,37000.00,37699.07,46229.447190,39753.245883,2562.147505,45762.240189,35769.523811,40765.8820,24.512450,0.0,0.0,45821.0,34322.28,40071.64,43062.906333,38701.426333,40882.166333,0.0,1.0,42.716844
2022-02-28,37699.08,44225.84,37450.17,43160.00,73945.638580,39887.329572,4668.908753,45616.042591,35827.422409,40721.7325,24.037828,0.0,0.0,45821.0,34322.28,40071.64,43137.640167,38381.193167,40759.416667,1.0,0.0,48.676201
